In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
import csv
import datetime
import os
import pytz
import smtplib
import email.message


options = Options()
options.add_experimental_option("detach", True)  
options.add_argument("--disable-dev-shm-usage")  # Melhora estabilidade
options.add_argument("--disable-extensions")  # Desativa extensões
options.add_argument("--disable-infobars")  # Remove barra de informações do Chrome
#options.add_argument("--headless")  # Modo headless (opcional)
service = Service(ChromeDriverManager().install())

data_hora0 = datetime.datetime.now()
data_hora00 = str(data_hora0)
print(f"Hora de início: {str(data_hora0)[0:16]}")

driver = webdriver.Chrome(service=service, options=options)

wait = WebDriverWait(driver, 20)
driver.maximize_window()

driver.get("https://plataformabrasil.saude.gov.br/login.jsf")
time.sleep(5)
print("Abrindo Plataforma Brasil")
titulo = driver.find_element(By.XPATH,'//*[@id="j_id27:esqueceuSenha"]').text
while titulo == 'Esqueceu a senha?':
    time.sleep(5)
    try:
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="formModalMensagemAviso:botaoFecharModal"]'))).click()
    except:
        pass

    driver.find_element(By.XPATH,'//*[@id="j_id19:email"]').clear() # email
    driver.find_element(By.XPATH,'//*[@id="j_id19:email"]').send_keys('gabrielcgs12@gmail.com') # email
    driver.find_element(By.XPATH,'//*[@id="j_id19:senha"]').clear() # senha
    driver.find_element(By.XPATH,'//*[@id="j_id19:senha"]').send_keys('0Dije!c!') # senha
       
    driver.find_element(By.XPATH, '//*[@id="j_id19"]/input[4]').click() # logar
    
    try:   
        driver.find_element(By.XPATH, '//*[@id="formModalMsgUsuarioLogado:idBotaoInvalidarUsuarioLogado"]').click()
    except:
        pass
    
    titulo = len((BeautifulSoup(driver.page_source, 'html.parser')).find("div",id="barraSistemaInf").text)
    driver.refresh()
print("Login realizado com sucesso")


Hora de início: 2025-02-24 09:55
Abrindo Plataforma Brasil


ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" name="j_id19:j_id24" value="LOGIN" alt="Clique aqui para acessar" onclick="Richfaces.showModalPanel('ajaxStatusMPLogin')" style="width:60px;" title="Clique aqui para acessar" class="buttomLoginDashboard" tabindex="0"> is not clickable at point (1811, 115). Other element would receive the click: <div class="rich-mpnl-mask-div-opaque rich-mpnl-mask-div" id="idModalBloqueioDiv" style="z-index: -1;">...</div>
  (Session info: chrome=133.0.6943.127)
Stacktrace:
	GetHandleVerifier [0x00FB0283+25139]
	(No symbol) [0x00F3B234]
	(No symbol) [0x00E104A3]
	(No symbol) [0x00E5E5D0]
	(No symbol) [0x00E5C984]
	(No symbol) [0x00E5A517]
	(No symbol) [0x00E597D4]
	(No symbol) [0x00E4DE55]
	(No symbol) [0x00E7CF3C]
	(No symbol) [0x00E4D8E4]
	(No symbol) [0x00E7D1B4]
	(No symbol) [0x00E9E6E1]
	(No symbol) [0x00E7CD36]
	(No symbol) [0x00E4BD29]
	(No symbol) [0x00E4D064]
	GetHandleVerifier [0x012BB143+3215603

In [ ]:
while True:
    try:
        elemento = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[3]/div[1]/strong')))
        print("Login bem-sucedido! Elemento encontrado:", elemento.text)
        driver.find_element(By.ID,"gerirPesquisaForm:idSituacoes:11:idItem").click()
        driver.find_element(By.ID,"gerirPesquisaForm:idSituacoes:14:idItem").click()
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gerirPesquisaForm:idBtnBuscarProjPesquisa"]'))).click()
        break
    except:
        print("Falha ao verificar o login.")
        time.sleep(1)

Login bem-sucedido! Elemento encontrado: Gabriel Calazans Gonçalves da Silva


In [12]:
while pesquisador != "":
    driver.find_element(By.ID,"gerirPesquisaForm:idSituacoes:11:idItem").click()
    driver.find_element(By.ID,"gerirPesquisaForm:idSituacoes:14:idItem").click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gerirPesquisaForm:idBtnBuscarProjPesquisa"]'))).click()
    break

In [ ]:
# baixar python 3.9
